In [60]:
import nltk
# import random
# import string
import re
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import Counter
from nltk.util import ngrams
corpus = []

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
with open('/content/dataset_3.txt', 'r') as file:
    # Read the contents of the file
    data = file.read()

# Now you can perform data cleaning operations on the 'data' variable
# For example, you can split the text into lines
lines = data.split('\n')

In [48]:
ps = PorterStemmer()
all_stopwords = set(stopwords.words("english"))
all_stopwords.remove("not")

reviews = data.split("\n")

for review  in reviews:
    review = re.sub("[a^zA-Z]", " ", review)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in all_stopwords]
    review = " ".join(review)
    corpus.append(review)

In [49]:
nltk.download('punkt')
token = nltk.word_tokenize(data)
token

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['1',
 '“',
 '18',
 'months',
 'ago',
 ',',
 'we',
 'expelled',
 'a',
 'boy',
 'at',
 'Nations',
 'for',
 'selling',
 'drugs',
 'in',
 'six',
 'schools',
 '.',
 '2',
 '”',
 '41',
 'Nigeria',
 'Centre',
 'for',
 'Disease',
 'Control',
 '(',
 'NCDC',
 ')',
 'staff',
 'and',
 '17',
 'World',
 'Health',
 'Organisation',
 '(',
 'WHO',
 ')',
 'staff',
 'are',
 'deployed',
 'at',
 'the',
 'moment',
 'to',
 'support',
 'the',
 'Kano',
 'response',
 '.',
 '3',
 '⏰8.00pm',
 '⚽️Liverpool',
 'v',
 'Arsenal',
 'Watch',
 'the',
 'match',
 'live',
 'at',
 'The',
 'Arch',
 'on',
 'six',
 'screens',
 'with',
 'surround',
 'sound',
 'commentary',
 '!',
 '4',
 'A',
 '13-year',
 'veteran',
 'of',
 'the',
 'department',
 ',',
 'he',
 'worked',
 'his',
 'way',
 'up',
 'the',
 'ranks',
 'of',
 'the',
 'department',
 'starting',
 'as',
 'an',
 'ambulance',
 'paramedic',
 'at',
 'Station',
 '49',
 ',',
 'then',
 'going',
 'to',
 'the',
 'SFFD',
 'Academy',
 'and',
 'graduating',
 'as',
 'a',
 'paramedic',
 'an

In [52]:
from sklearn.model_selection import train_test_split

train_tokens, val_tokens = train_test_split(token, test_size = 0.3, random_state = 42)

In [63]:
from collections import defaultdict

def build_bigram_model(tokens):
    bigram_model = defaultdict(lambda: defaultdict(int))

    for i in range(len(tokens) - 1):
        current_word, next_word = tokens[i], tokens[i + 1]
        bigram_model[current_word][next_word] += 1

    return bigram_model

def build_trigram_model(tokens):
    trigram_model = defaultdict(lambda: defaultdict(int))

    for i in range(len(tokens) - 2):
        current_words, next_word = (tokens[i], tokens[i + 1]), tokens[i + 2]
        trigram_model[current_words][next_word] += 1

    return trigram_model

# Example usage
tokens = ["I", "love", "to", "eat", "apples", "and", "bananas"]
bigram_model = build_bigram_model(tokens)
trigram_model = build_trigram_model(tokens)

# Print the models (for demonstration purposes)
print("Bigram Model:")
for word, next_words in bigram_model.items():
    print(f"{word}: {dict(next_words)}")

print("\nTrigram Model:")
for word_pair, next_words in trigram_model.items():
    print(f"{word_pair}: {dict(next_words)}")


Bigram Model:
I: {'love': 1}
love: {'to': 1}
to: {'eat': 1}
eat: {'apples': 1}
apples: {'and': 1}
and: {'bananas': 1}

Trigram Model:
('I', 'love'): {'to': 1}
('love', 'to'): {'eat': 1}
('to', 'eat'): {'apples': 1}
('eat', 'apples'): {'and': 1}
('apples', 'and'): {'bananas': 1}


In [64]:
def build_bigram_model_with_smoothing(tokens, vocab_size):
    bigram_model = defaultdict(lambda: defaultdict(lambda: 1))  # Start counts at 1 for smoothing

    for i in range(len(tokens) - 1):
        current_word, next_word = tokens[i], tokens[i + 1]
        bigram_model[current_word][next_word] += 1

    # Apply Laplace smoothing
    for current_word in bigram_model.keys():
        for next_word in bigram_model[current_word].keys():
            bigram_model[current_word][next_word] += 1
        bigram_model[current_word]["<UNK>"] = 1  # Unknown words

    return bigram_model

def build_trigram_model_with_smoothing(tokens, vocab_size):
    trigram_model = defaultdict(lambda: defaultdict(lambda: 1))  # Start counts at 1 for smoothing

    for i in range(len(tokens) - 2):
        current_words, next_word = (tokens[i], tokens[i + 1]), tokens[i + 2]
        trigram_model[current_words][next_word] += 1

    # Apply Laplace smoothing
    for current_words in trigram_model.keys():
        for next_word in trigram_model[current_words].keys():
            trigram_model[current_words][next_word] += 1
        trigram_model[current_words]["<UNK>"] = 1  # Unknown words

    return trigram_model

# Example usage
vocab_size = len(set(tokens))  # Assuming tokens is a list of all words in the dataset
bigram_model_smoothed = build_bigram_model_with_smoothing(tokens, vocab_size)
trigram_model_smoothed = build_trigram_model_with_smoothing(tokens, vocab_size)

# Print the smoothed models (for demonstration purposes)
print("Bigram Model with Laplace Smoothing:")
for word, next_words in bigram_model_smoothed.items():
    print(f"{word}: {dict(next_words)}")

print("\nTrigram Model with Laplace Smoothing:")
for word_pair, next_words in trigram_model_smoothed.items():
    print(f"{word_pair}: {dict(next_words)}")


Bigram Model with Laplace Smoothing:
I: {'love': 3, '<UNK>': 1}
love: {'to': 3, '<UNK>': 1}
to: {'eat': 3, '<UNK>': 1}
eat: {'apples': 3, '<UNK>': 1}
apples: {'and': 3, '<UNK>': 1}
and: {'bananas': 3, '<UNK>': 1}

Trigram Model with Laplace Smoothing:
('I', 'love'): {'to': 3, '<UNK>': 1}
('love', 'to'): {'eat': 3, '<UNK>': 1}
('to', 'eat'): {'apples': 3, '<UNK>': 1}
('eat', 'apples'): {'and': 3, '<UNK>': 1}
('apples', 'and'): {'bananas': 3, '<UNK>': 1}


In [67]:
def calculate_sentence_probability(sentence, model):
    words = sentence.split()
    probability = 1.0

    # Calculate probability using the bigram model
    if len(words) < 2:
        return probability

    for i in range(len(words) - 1):
        current_word, next_word = words[i], words[i + 1]
        if current_word in model and next_word in model[current_word]:
            transition_probability = model[current_word][next_word] / sum(model[current_word].values())
            probability *= transition_probability
        else:
            # Handle unknown words using Laplace smoothing
            numerator = model[current_word].get(next_word, 0) + 1  # Add 1 for Laplace smoothing
            denominator = sum(model[current_word].values()) + len(model[current_word])  # Add vocabulary size for Laplace smoothing
            if denominator != 0:  # Avoid division by zero
                transition_probability = numerator / denominator
                probability *= transition_probability

    return probability

# Example usage
test_sentence = "This is a test sentence."
bigram_probability = calculate_sentence_probability(test_sentence, bigram_model_smoothed)
trigram_probability = calculate_sentence_probability(test_sentence, trigram_model_smoothed)

print("Probability of the sentence using Bigram Model with Laplace Smoothing:", bigram_probability)
print("Probability of the sentence using Trigram Model with Laplace Smoothing:", trigram_probability)


Probability of the sentence using Bigram Model with Laplace Smoothing: 1.0
Probability of the sentence using Trigram Model with Laplace Smoothing: 1.0


In [68]:
def evaluate_models(test_data, bigram_model, trigram_model):
    correct_bigram_predictions = 0
    correct_trigram_predictions = 0
    total_sentences = len(test_data)

    for sentence in test_data:
        # Calculate probability for bigram model
        bigram_probability = calculate_sentence_probability(sentence, bigram_model_smoothed)

        # Calculate probability for trigram model
        trigram_probability = calculate_sentence_probability(sentence, trigram_model_smoothed)

        # Determine which model has higher probability
        if bigram_probability > trigram_probability:
            correct_bigram_predictions += 1
        elif bigram_probability < trigram_probability:
            correct_trigram_predictions += 1
        else:
            # If probabilities are equal, we can choose any model
            correct_bigram_predictions += 1

    # Calculate accuracy
    bigram_accuracy = (correct_bigram_predictions / total_sentences) * 100
    trigram_accuracy = (correct_trigram_predictions / total_sentences) * 100

    return bigram_accuracy, trigram_accuracy

# Evaluate the models
bigram_accuracy, trigram_accuracy = evaluate_models(test_data, bigram_model_smoothed, trigram_model_smoothed)

# Print the accuracy of each model
print("Bigram Model Accuracy:", bigram_accuracy)
print("Trigram Model Accuracy:", trigram_accuracy)


Bigram Model Accuracy: 100.0
Trigram Model Accuracy: 0.0
